In [1]:
!pip install pulp
from pulp import *

In [2]:
# A dictionary of the profits per cocktail
profit = {'Daiquiri': 6, 
          'Kamikaze': 5,
          'LongIsland': 7}

# A dictionary of the the alcohols per cocktail
portion = {'Daiquiri':
            {'whiteRum': 50,
            'cointreau': 30,
            'gin': 0,
            'vodka': 0
            },
            'Kamikaze':
            {'whiteRum': 0,
            'cointreau': 30,
            'gin': 0,
            'vodka': 40
            },
            'LongIsland':
            {'whiteRum': 20,
            'cointreau': 20,
            'gin': 20,
            'vodka': 20
            }
          }

# A dictionary of the available per alcohols
alcohols = {'whiteRum': 5000,
            'cointreau': 7000,
            'gin': 3000,
            'vodka': 6000
            }

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Bartender Problem", LpMaximize)

# A dictionary called 'production_vars' is created to contain the referenced Variables
x = LpVariable.dicts("Cocktails", profit.keys(), 0)

# Create objective function
prob += lpSum([profit[c]*x[c] for c in profit.keys()])

# Create constraints
for a in alcohols.keys():
    prob += lpSum([x[c]*portion[c][a] for c in profit.keys()]) <= alcohols[a], "{}".format(a)

In [3]:
print(prob)

Bartender Problem:
MAXIMIZE
6*Cocktails_Daiquiri + 5*Cocktails_Kamikaze + 7*Cocktails_LongIsland + 0
SUBJECT TO
whiteRum: 50 Cocktails_Daiquiri + 20 Cocktails_LongIsland <= 5000

cointreau: 30 Cocktails_Daiquiri + 30 Cocktails_Kamikaze
 + 20 Cocktails_LongIsland <= 7000

gin: 20 Cocktails_LongIsland <= 3000

vodka: 40 Cocktails_Kamikaze + 20 Cocktails_LongIsland <= 6000

VARIABLES
Cocktails_Daiquiri Continuous
Cocktails_Kamikaze Continuous
Cocktails_LongIsland Continuous



In [5]:
prob.writeLP("../Data/Bartender.lp")

In [6]:
# Optimize
prob.solve()

# Print the status of the solved LP
print("Status = %s" % LpStatus[prob.status])

Status = Optimal


In [7]:
# Print the value of the variables at the optimum
for c in profit:
    print("%s = %f" % (x[c].name, x[c].varValue))

# Print the value of the objective
print("Objective = %f" % value(prob.objective))

Cocktails_Daiquiri = 40.000000
Cocktails_Kamikaze = 75.000000
Cocktails_LongIsland = 150.000000
Objective = 1665.000000


In [8]:
for a in alcohols:
    print("Shadow Price | Slack of %s: %f | %f" %
          (a, prob.constraints[a].pi, prob.constraints[a].slack))
    
for p in profit:
    print("Reduced Cost of %s: %f" % (p, x[p].dj))

Shadow Price | Slack of whiteRum: 0.120000 | -0.000000
Shadow Price | Slack of cointreau: -0.000000 | 550.000000
Shadow Price | Slack of gin: 0.105000 | -0.000000
Shadow Price | Slack of vodka: 0.125000 | -0.000000
Reduced Cost of Daiquiri: 0.000000
Reduced Cost of Kamikaze: 0.000000
Reduced Cost of LongIsland: 0.000000
